In [1]:
import mlflow


In [3]:
mlflow.set_tracking_uri("http://nodelogin02:5000")
experiment = mlflow.set_experiment("lung_fun_db15")

# Create a run under the default experiment (whose id is '0').
client = mlflow.MlflowClient()
for i in range(1503, 1504):
    experiment_id = i
    run_ls = client.search_runs(experiment_ids=[experiment.experiment_id], run_view_type=1, filter_string=f"params.id LIKE '%{i}%'")
    if len(run_ls)!=0:
        run_ = run_ls[0]
        run_id = run_.info.run_id
        with mlflow.start_run(run_id=run_id):
            mlflow.log_param("sub_shuffle", 'True')
            mlflow.log_param("total_shuffle", 'True')


